In [0]:
! pip install kmc2

     |████████████████████████████████| 102kB 2.3MB/s 
     |████████████████████████████████| 163kB 8.2MB/s 
  Created wheel for kmc2: filename=kmc2-0.1-cp36-cp36m-linux_x86_64.whl size=252255 sha256=051d626e96ee7002e7c6ebb658ca4193ecfbc6b4f9cf1fcfe57be5ee86c162a0
  Stored in directory: /root/.cache/pip/wheels/5c/ba/f0/4c8b421be72d4f2d1a93233c2f6f591e7d8b0bda05a1f4616f
Successfully built kmc2


In [0]:
! yes | pip3 uninstall opencv-python
! yes | pip3 uninstall opencv-contrib-python
! yes | pip3 install opencv-python==3.4.2.16
! yes | pip3 install opencv-contrib-python==3.4.2.16

Uninstalling opencv-python-4.1.2.30:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/cv2/*
    /usr/local/lib/python3.6/dist-packages/opencv_python-4.1.2.30.dist-info/*
Proceed (y/n)?   Successfully uninstalled opencv-python-4.1.2.30
Uninstalling opencv-contrib-python-4.1.2.30:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/opencv_contrib_python-4.1.2.30.dist-info/*
Proceed (y/n)?   Successfully uninstalled opencv-contrib-python-4.1.2.30
     |████████████████████████████████| 25.0MB 390kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 30.6MB 348kB/s 


In [0]:
from sklearn.decomposition import MiniBatchDictionaryLearning
from sklearn.decomposition import DictionaryLearning
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import SparseCoder
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from imutils import paths

import matplotlib.pyplot as plt
import scipy.cluster.vq as vq
import _pickle as cPickle
import pandas as pd
import numpy as np
import imutils 
import torch
import time
import math
import copy
import kmc2
import cv2 
import os

## 데이터 불러오기

In [0]:
!ls -lha kaggle.json

!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

-rw-r--r-- 1 root root 65 Dec 11 14:27 kaggle.json
Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.3.1)
     |████████████████████████████████| 61kB 2.1MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-cp36-none-any.whl size=72859 sha256=c7846c989375f23c571eba3261cff55720baa456a27104beb13ea4384bb0c101
  Stored in directory: /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674
Successfully built kaggle


In [0]:
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

! kaggle -v

Kaggle API 1.5.6


In [0]:
! kaggle competitions download -c 2019-ml-finalproject
! A | unzip 2019-ml-finalproject.zip

 62% 40.0M/65.0M [00:00<00:00, 193MB/s]
100% 65.0M/65.0M [00:00<00:00, 234MB/s]
/bin/bash: A: command not found
Archive:  2019-ml-finalproject.zip
  inflating: Label2Names.csv         
  inflating: sample_submission.csv   
  inflating: testAll_v2/image_0001.jpg  
  inflating: testAll_v2/image_0002.jpg  
  inflating: testAll_v2/image_0003.jpg  
  inflating: testAll_v2/image_0004.jpg  
  inflating: testAll_v2/image_0005.jpg  
  inflating: testAll_v2/image_0006.jpg  
  inflating: testAll_v2/image_0007.jpg  
  inflating: testAll_v2/image_0008.jpg  
  inflating: testAll_v2/image_0009.jpg  
  inflating: testAll_v2/image_0010.jpg  
  inflating: testAll_v2/image_0011.jpg  
  inflating: testAll_v2/image_0012.jpg  
  inflating: testAll_v2/image_0013.jpg  
  inflating: testAll_v2/image_0014.jpg  
  inflating: testAll_v2/image_0015.jpg  
  inflating: testAll_v2/image_0016.jpg  
  inflating: testAll_v2/image_0017.jpg  
  inflating: testAll_v2/image_0018.jpg  
  inflating: testAll_v2/image_0019.jpg 

In [0]:
caltech_dir = "/content/train/"
lists = paths.list_images(caltech_dir)

In [0]:
X = []
y = []

for l in lists:
  path = os.path.dirname(l)
  # if l[15] != 'B':
  X.append(cv2.imread(l))
  y.append(os.path.basename(path))

# categories.remove('BACKGROUND_Google') # it contains all kinds of images. 
print(len(X))

3060


In [0]:
X = np.array(X)
y = np.array(y)

print(X.shape)
print(y.shape)

(3060,)
(3060,)


In [0]:
def describeSURF( image):
    surf = cv2.xfeatures2d.SURF_create()
    surf.setHessianThreshold(400)
    kp, des = surf.detectAndCompute(image,None)
    return kp,des

In [0]:
def describeSIFT( image):
    sift = cv2.xfeatures2d.SIFT_create()
    kp, des = sift.detectAndCompute(image,None)
    return kp,des

In [0]:
def describeORB( image):
    orb=cv2.ORB_create()
    kp, des=orb.detectAndCompute(image,None)
    return kp,des

In [0]:
import itertools

def getDescriptors(X):
    descriptors=list()
    
    for image in X:
        kp, des = describeSIFT(image)
        descriptors.append(des)
        # print(len(kp))
        
    #flatten list       
    descriptors = list(itertools.chain.from_iterable(descriptors))
    #list to array
    descriptors = np.asarray(descriptors)

    return descriptors

In [0]:
descriptors = getDescriptors(X)

In [0]:
with open('/content/descriptorORB.pickle', 'wb') as f:
	  cPickle.dump(descriptors, f)

In [0]:
with open('/content/descriptorORB.pickle', 'rb') as f:
    descriptors=cPickle.load(f)

In [0]:
def kMeansDictionary(training, k):
    seeding = kmc2.kmc2(training, k)
    est = MiniBatchKMeans(k, init=seeding, tol=0.0001, verbose=1).fit(training)
    # est = KMeans(n_clusters=k,init='k-means++',tol=0.0001,verbose=1).fit(training)

    return est

In [0]:
visualDictionary = kMeansDictionary(descriptors, 200)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: Explicit initial center position passed: performing only one init in MiniBatchKMeans instead of n_init=3
  This is separate from the ipykernel package so we can avoid doing imports until


Init 1/1 with method: [[36.  4.  1. ...  0.  0.  0.]
 [ 3.  5. 10. ...  0.  0.  0.]
 [19.  3.  1. ... 49. 89. 32.]
 ...
 [ 6.  1.  3. ... 60. 41.  9.]
 [10. 26. 86. ...  0.  0.  0.]
 [ 0.  1.  4. ...  3.  4. 14.]]
Inertia for init 1/1: 14176782.000000
Minibatch iteration 1/1382700: mean batch inertia: 104636.140000, ewa inertia: 104636.140000 
Minibatch iteration 2/1382700: mean batch inertia: 101224.830000, ewa inertia: 104635.646554 
Minibatch iteration 3/1382700: mean batch inertia: 97350.560000, ewa inertia: 104634.592766 
Minibatch iteration 4/1382700: mean batch inertia: 94210.500000, ewa inertia: 104633.084922 
Minibatch iteration 5/1382700: mean batch inertia: 89350.740000, ewa inertia: 104630.874331 
Minibatch iteration 6/1382700: mean batch inertia: 95816.750000, ewa inertia: 104629.599368 
Minibatch iteration 7/1382700: mean batch inertia: 94575.850000, ewa inertia: 104628.145093 
Minibatch iteration 8/1382700: mean batch inertia: 95126.320000, ewa inertia: 104626.770655 
Mi

In [0]:
with open('/content/visualDictionary2ORB.pickle', 'wb') as f:
	cPickle.dump(visualDictionary, f)

In [0]:
with open('/content/visualDictionary2ORB.pickle', 'rb') as f:
    visualDictionary=cPickle.load(f) 

In [0]:
def VLAD(X,visualDictionary):

    predictedLabels = visualDictionary.predict(X)
    centers = visualDictionary.cluster_centers_
    labels=visualDictionary.labels_
    k=visualDictionary.n_clusters
   
    m,d = X.shape
    V=np.zeros([k,d])
    #computing the differences

    # for all the clusters (visual words)
    for i in range(k):
        # if there is at least one descriptor in that cluster
        if np.sum(predictedLabels==i)>0:
            # add the diferences
            V[i]=np.sum(X[predictedLabels==i,:]-centers[i],axis=0)
    

    V = V.flatten()
    # power normalization, also called square-rooting normalization
    V = np.sign(V)*np.sqrt(np.abs(V))

    # L2 normalization

    V = V/np.sqrt(np.dot(V,V))
    return V

In [0]:
def getVLADDescriptors(X,visualDictionary):
    descriptors=list()
    # idImage =list()
    for img in X:
        kp,des = describeSIFT(img)
        v = VLAD(des,visualDictionary)
        descriptors.append(v)
        # idImage.append(imagePath)
                    
    #list to array    
    descriptors = np.asarray(descriptors)
    return descriptors # , idImage

In [0]:
V = getVLADDescriptors(X, visualDictionary)

Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing 

In [0]:
with open('/content/VLAD_ORB_W2.pickle', 'wb') as f:
	cPickle.dump(V, f)

In [0]:
with open('/content/VLAD_ORB_W2.pickle', 'rb') as f:
    VLAD_DS = cPickle.load(f) 

In [0]:
all_train_desc = np.vstack((descriptor for descriptor in VLAD_DS))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """Entry point for launching an IPython kernel.


In [0]:
clf = LinearSVC(random_state=0, C=0.001)
clf.fit(all_train_desc, y)

LinearSVC(C=0.001, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
          verbose=0)

In [0]:
caltech_dir = "/content/testAll_v2/"
lists = paths.list_images(caltech_dir)

X_final = []
img_name = []
for l in lists:
  path = os.path.dirname(l)
  X_final.append(cv2.imread(l))
  img_name.append(l[-14:])

X_final = np.array(X_final)

In [0]:
f_descriptors = getDescriptors(X_final)

In [0]:
f_visualDictionary = kMeansDictionary(f_descriptors, 200)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: Explicit initial center position passed: performing only one init in MiniBatchKMeans instead of n_init=3
  This is separate from the ipykernel package so we can avoid doing imports until


Init 1/1 with method: [[ 7.  2.  2. ...  0.  2.  2.]
 [ 3. 25. 33. ... 36. 62. 23.]
 [ 0.  0.  3. ...  0.  0.  0.]
 ...
 [31.  5.  5. ... 22. 18. 97.]
 [ 2.  1.  3. ... 49.  2.  4.]
 [ 2.  0.  0. ...  0.  0.  0.]]
Inertia for init 1/1: 12876855.000000
Minibatch iteration 1/762200: mean batch inertia: 106015.640000, ewa inertia: 106015.640000 
Minibatch iteration 2/762200: mean batch inertia: 104996.640000, ewa inertia: 106015.372591 
Minibatch iteration 3/762200: mean batch inertia: 98769.150000, ewa inertia: 106013.471017 
Minibatch iteration 4/762200: mean batch inertia: 95674.680000, ewa inertia: 106010.757883 
Minibatch iteration 5/762200: mean batch inertia: 94023.680000, ewa inertia: 106007.612201 
Minibatch iteration 6/762200: mean batch inertia: 93233.410000, ewa inertia: 106004.259960 
Minibatch iteration 7/762200: mean batch inertia: 86498.410000, ewa inertia: 105999.141181 
Minibatch iteration 8/762200: mean batch inertia: 93537.200000, ewa inertia: 105995.870884 
Minibatch 

In [0]:
f_VLAD_DS = getVLADDescriptors(X_final, f_visualDictionary)

Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing label assignment and total inertia
Computing 

In [0]:
f_all_train_desc = np.vstack((descriptor for descriptor in f_VLAD_DS))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """Entry point for launching an IPython kernel.


In [0]:
y_predict = clf.predict(f_all_train_desc)

In [0]:
df = pd.read_csv('/content/Label2Names.csv')
df = np.array(df)

result = []

for i in range(len(y_predict)):
  flag = 0
  if y_predict[i] == 'BACKGROUND_Google':
    flag = 1
    result.append(101)
  else:
    for j in range(len(df)):
        if y_predict[i] == df[j][1]:
          result.append(df[j][0])
          flag = 1
  
  if flag != 1:
    result.append(1)

result = np.array(result)
img_name = np.array(img_name)
result = result.reshape(-1, 1)
img_name = img_name.reshape(-1, 1)
total_result = np.hstack([img_name, result])
print(total_result)

[['image_1073.jpg' '45']
 ['image_0436.jpg' '93']
 ['image_0181.jpg' '93']
 ...
 ['image_0319.jpg' '44']
 ['image_1096.jpg' '20']
 ['image_0608.jpg' '45']]


In [0]:
df = pd.DataFrame(total_result, columns=['Id', 'Category'])
df.to_csv('results-mjkim-v4.csv', index=False, header=True)

In [0]:
! kaggle competitions submit -c 2019-ml-finalproject -f results-mjkim-v4.csv -m "mjkim-20191123"

100% 29.6k/29.6k [00:06<00:00, 4.46kB/s]
